In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

/opt/conda/envs/project2/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/opt/ml/level2_objectdetection-cv-11/UniverseNet/configs/tood/tood_swinv2-t-p4-w16_fpn_resize_coco_adamw.py')

root='../../dataset/'

epoch = 'best_bbox_mAP_50_epoch_20'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + args.trainset # train json 정보
if "dataset" in cfg.data.train.keys():
    cfg.data.train.dataset.pipeline[2]['img_scale'] = (resize,resize)
else:
    cfg.data.train.pipeline[2]['img_scale'] = (resize,resize) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + args.validset # valid json 정보
cfg.data.val.pipeline[1]['img_scale'] = (resize,resize) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = '/opt/ml/level2_objectdetection-cv-11/UniverseNet/work_dirs/tood_swinv2-t-p4-w16_fpn_resize_coco_adamw_trash'

if "roi_head" in cfg.model.keys():
        if type(cfg.model.roi_head.bbox_head) == dict:
            cfg.model.roi_head.bbox_head.num_classes = 10

        #In case of cascade RCNN : List[Dict]
        elif type(cfg.model.roi_head.bbox_head) == list:
            for each_head in cfg.model.roi_head.bbox_head:
                if hasattr(each_head, "num_classes"):
                    each_head.num_classes = 10 
                else: 
                    raise Exception("Num_classes가 없습니다")
else:
    cfg.model.bbox_head.num_classes = 10
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

NameError: name 'args' is not defined

In [ ]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [ ]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: /opt/ml/level2_objectdetection-cv-11/UniverseNet/work_dirs/tood_swinv2-t-p4-w16_fpn_resize_coco_trash/best_bbox_mAP_50_epoch_20.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: backbone.layers.0.blocks.0.attn.relative_coords_table, backbone.layers.0.blocks.0.attn.relative_position_index, backbone.layers.0.blocks.0.attn.k_bias, backbone.layers.0.blocks.1.attn.relative_coords_table, backbone.layers.0.blocks.1.attn.relative_position_index, backbone.layers.0.blocks.1.attn.k_bias, backbone.layers.1.blocks.0.attn.relative_coords_table, backbone.layers.1.blocks.0.attn.relative_position_index, backbone.layers.1.blocks.0.attn.k_bias, backbone.layers.1.blocks.1.attn.relative_coords_table, backbone.layers.1.blocks.1.attn.relative_position_index, backbone.layers.1.blocks.1.attn.k_bias, backbone.layers.2.blocks.0.attn.relative_coords_table, backbone.layers.2.blocks.0.attn.relative_position_index, backbone.layers.2.blocks

In [ ]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 977/977, 10.1 task/s, elapsed: 97s, ETA:     0s

In [ ]:
coco.getAnnIds()

[18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 175,
 176,
 177,
 178,
 229,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 247,
 248,
 249,
 250,
 287,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 331,
 332,
 355,
 356,
 357,
 358,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 383,
 424,
 425,
 431,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 479,
 480,
 481,
 482,
 483,
 484,
 485,
 486,
 487,
 488,
 489,
 490,
 491,
 492,
 493,
 494,
 495,
 496,
 497,
 498,
 499,
 500,
 535,
 536,
 537,
 538,
 539,
 540,
 541,
 550,
 551,
 552,
 553,
 554,
 555,
 556,
 557,
 558,
 559,
 560,
 561,
 562,
 563,
 564,
 565,
 566,
 567,
 568,
 569,
 570,
 571,
 572,
 

In [ ]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in zip(img_ids, output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}_val.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.35769778 908.2944 164.62697 1024.2551 388....,train/0005.jpg
1,0 0.4286269 -0.0063753948 267.89792 135.5214 4...,train/0013.jpg
2,0 0.25513017 272.00302 355.0783 392.38882 379....,train/0019.jpg
3,0 0.43773863 134.9265 459.11328 222.61018 523....,train/0038.jpg
4,0 0.58418816 369.3948 462.8648 677.8683 650.95...,train/0041.jpg
